In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
os.chdir(r"C:\Users\sapir\out_of_onedrive\dina\CalibVEP")

In [5]:
import utils
import definitions as d
import torch
from data_classes import METHODS_TO_ESM, MutationVariantSet
from mutation_record import MutationRecord
from utils import is_disordered
from load_cluster_lookup import get_cluster_sizes_of_sequence


c:\Users\sapir\anaconda3\envs\env_image_processing\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import pickle

import numpy as np

        



def get_pathogenic_percentage(tree_path_key: str, leaf_best_scores: dict, bins_dict: dict) -> tuple[str, float]:
    """Get the pathogenic percentage for a mutation based on its tree path key.

    Args:
        tree_path_key (str): The key representing the tree path for the mutation.
        leaf_best_scores (dict): A dictionary containing the best scores for each tree path.
        bins_dict (dict): A dictionary containing bin information for each tree path.

    Returns:
        tuple[str, float]: A tuple containing the bin index as a string and the pathogenic percentage.
    """
    bin_key, score_llr = leaf_best_scores[tree_path_key]

    # Get the bin information
    bin_info = bins_dict[tree_path_key][bin_key]
    bin_edges = np.array(bin_info['bin_edges'])
    bin_stats = bin_info['bin_stats']

    # Find which bin the score falls into
    bin_index = np.digitize(score_llr, bin_edges) - 1

    # Handle edge cases
    if bin_index < 0:
        bin_index = 0
    elif bin_index >= len(bin_edges) - 1:
        bin_index = len(bin_edges) - 2

    # Convert bin_index to string (as that's how it's stored in bin_stats)
    bin_index_str = str(bin_index)
    pathogenic_percentage = bin_stats[int(bin_index_str)]['path_pct']

    return score_llr, pathogenic_percentage


In [17]:
import pandas as pd
# df = pd.read_parquet(r"c:\Users\sapir\out_of_onedrive\dina\entropy-missense-prediction\resources\2_sort_runnings\preprocessed_P53_HUMAN.parquet")
df = pd.read_parquet(r"C:\Users\sapir\out_of_onedrive\dina\entropy-missense-prediction\resources\2_long_proteins\preprocessed_SCN1A_HUMAN.parquet")

In [22]:
# df.mutant = K2009S  Score LLR: -0.718459963798523, Pathogenic Percentage: 0.79%
df[df['mutant'] == 'K2009S']

,wt_aa,position,mt_aa,pathogenic_label,Gold_Stars,NumberSubmitters,Starred_Coarse_Grained_Clin_Sig,frequency_gv2,frequency_gv3,evolutionary_index_ASM,...,lp_acmg_model,coarse_clinical_significance_post,clinical_significance_post,model_disagreement,CV_label_and_model_combined_with_other_evidence_disagreement,starred_label_and_model_disagreement,starred_label_and_model_combined_with_other_evidence_disagreement,mutant,protein_sequence,protein_name
40175,K,2009,S,None,NaN,NaN,None,NaN,NaN,NaN,...,False,None,None,False,False,False,False,K2009S,MEQTVLVPPGPDSFNFFTRESLAAIERRIAEEKAKNPKPDKKDDDE...,SCN1A_HUMAN


In [9]:
seq = "MEEPQSDPSVEPPLSQETFSDLWKLLPENNVLSPLPSQAMDDLMLSPDDIEQWFTEDPGPDEAPRMPEAAPPVAPAPAAPTPAAPAPAPSWPLSSSVPSQKTYQGSYGFRLGFLHSGTAKSVTCTYSPALNKMFCQLAKTCPVQLWVDSTPPPGTRVRAMAIYKQSQHMTEVVRRCPHHERCSDSDGLAPPQHLIRVEGNLRVEYLDDRNTFRHSVVVPYEPPEVGSDCTTIHYNYMCNSSCMGGMNRRPILTIITLEDSSGNLLGRNSFEVRVCACPGRDRRTEEENLRKKGEPHHELPPGSTKRALPNNTSSSPQPKKKPLDGEYFTLQIRGRERFEMFRELNEALELKDAQAGKEPGGSRAHSSHLKSKKGQSTSRHKKLMFKTEGPDSD"
mutation = "M1A"
# esm_model_name = "esm1b_t33_650M_UR50S"
esm_model_name = "esm1_t6_43M_UR50S"

In [10]:
orig_model, orig_alphabet = utils.esm_setup(esm_model_name, device=d.DEVICE)

Using cache found in C:\Users\sapir/.cache\torch\hub\facebookresearch_esm_main


model loaded on cpu


In [ ]:
mutation_variant_set = run_esm(orig_model, orig_alphabet, [33], seq, mutation, esm_model_name)

leaf_best_scores = {
    'all/homologs_0_to_450/disordered/longer_than_1022': ['wt_marginals_base_wt_score_126', mutation_variant_set.wt.nadav_base_wt_score.item()],
    'all/homologs_0_to_450/disordered/shorter_than_1022': ['masked_marginals_entropy_weighted_llr_score_142', mutation_variant_set.masked.entropy_weighted_llr_score.item()],
    'all/homologs_0_to_450/ordered/longer_than_1022': ['wt_marginals_base_wt_score_200', mutation_variant_set.wt.nadav_base_wt_score.item()],
    'all/homologs_0_to_450/ordered/shorter_than_1022': ['mutant_marginals_entropy_weighted_llr_score_200', mutation_variant_set.mutante.entropy_weighted_llr_score.item()],
    'all/homologs_450_plus/disordered/longer_than_1022': ['wt_not_nadav_marginals_base_wt_score_203', mutation_variant_set.wt.llr_base_score.item()],
    'all/homologs_450_plus/disordered/shorter_than_1022': ['wt_not_nadav_marginals_base_wt_score_77', mutation_variant_set.wt.llr_base_score.item()],
    'all/homologs_450_plus/ordered/longer_than_1022': ['mutant_marginals_entropy_weighted_llr_score_200', mutation_variant_set.mutante.entropy_weighted_llr_score.item()],
    'all/homologs_450_plus/ordered/shorter_than_1022': ['masked_marginals_entropy_weighted_llr_score_200', mutation_variant_set.masked.entropy_weighted_llr_score.item()],
}

In [13]:
data_path = 'data/new_clusters_sequences.parquet'
# homolog_count = get_cluster_sizes_of_sequence(seq, 'data', 'parquet')

In [ ]:


aa_mut = utils.process_mutation_name(mutation, offset=1)
is_disorder_region = is_disordered(seq, aa_mut.mut_idx)
tree_path_key = utils.classify_mutation_to_tree_path(
    homolog_count=300,
    is_disordered=is_disorder_region,
    sequence_length=len(seq)
)

In [ ]:
with open(d.PATH_TO_BIN_DICT, "rb") as f:
    bins_dict = pickle.load(f)

score_llr, pathogenic_percentage = get_pathogenic_percentage(tree_path_key, leaf_best_scores, bins_dict)
print(f"Tree Path Key: {tree_path_key}")
print(f"Score LLR: {score_llr}, Pathogenic Percentage: {pathogenic_percentage:.2f}%")

Tree Path Key: all/homologs_0_to_450/disordered/shorter_than_1022
Score LLR: -4.033759117126465, Pathogenic Percentage: 52.82%


In [1]:
print(5)

5
